In [1]:
import keras
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from keras import optimizers
from keras.layers.advanced_activations import ELU, PReLU, LeakyReLU
from keras.layers import Dense, Dropout, Activation, Flatten

Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
model = Sequential()
model.add(Conv2D(8, (3, 3), padding='same',
                 input_shape=(513, 800, 3)))
model.add(Activation('relu'))
model.add(Conv2D(8, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(16, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(16, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.25))

model.add(Flatten())
#model.add(Dense(10))
model.add(Activation('relu'))
#model.add(Dropout(0.5))
model.add(Dense(57))
model.add(Activation('softmax'))

model.summary()

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 513, 800, 8)       224       
_________________________________________________________________
activation_1 (Activation)    (None, 513, 800, 8)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 511, 798, 8)       584       
_________________________________________________________________
activation_2 (Activation)    (None, 511, 798, 8)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 255, 399, 8)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 255, 399, 8)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 255, 399, 16)      1168      
__________

In [3]:
model.load_weights('my_model_weights.h5')

## Import data

In [4]:
import subprocess
import matplotlib.pyplot
import os
import matplotlib.pyplot as plt
rootdir = '/home/ubuntu/t_learning/'

In [5]:
spectograms = []
spect_read = []
spectograms_ids = []
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if file.endswith('png'): 
            try:
                x = plt.imread(subdir+'/'+file)
            except:
                continue
            if str(x.shape) == '(513, 800, 3)': 
                spect_read.append(x)
                name = subdir.replace(rootdir, '')
                name = name.replace('/spects', "")
                spectograms_ids.append(name)
                spectograms.append(file)
                print(name)
x = spect_read
y = spectograms_ids

Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar
Tushar

Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj
Sriraj


## Create training and test sets

In [6]:
import random
test_indices = random.sample(range(len(x)), 50)
train_indices = []
for i in range(len(x)):
    if i not in test_indices:
        train_indices.append(i)

x_new_train = []
y_new_train = []
x_test = []
y_test = []

for i in test_indices:
    x_test.append(x[i])
    y_test.append(y[i])


for i in train_indices:
    x_new_train.append(x[i])
    y_new_train.append(y[i])

In [7]:
import keras
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

### One hot encode

In [10]:
encoder = LabelEncoder()
y_temp_train = y_new_train
encoder.fit(y_temp_train)
encoded_Y = encoder.transform(y_temp_train)
dummy_y = np_utils.to_categorical(encoded_Y)

y_temp2_train = y_test
encoder.fit(y_temp2_train)
encoded_Y = encoder.transform(y_temp2_train)
dummy2_y = np_utils.to_categorical(encoded_Y)

# Transfer Learning 

In [11]:
dummy2_y.shape

(50, 3)

In [12]:
#from keras.applications.vgg19 import preprocess_input
from keras.models import Model
import numpy as np

In [13]:
model2 = Model(inputs=model.input, outputs=model.get_layer('flatten_1').output)

### Create dataset for SVM based on NN features

In [14]:
svm_x2_train = []
svm_y2_train = []
for i in range(len(x_new_train)):
    x_1 = np.expand_dims(x_new_train[i], axis=0)
    #x_1 = preprocess_input(x_1)
    flatten_2_features = model2.predict(x_1)
    svm_x2_train.append(flatten_2_features)
    svm_y2_train.append(dummy_y[i])

In [15]:
svm_x_train = []
svm_y_train = []
test_indices = random.sample(range(len(svm_x2_train)), 50)
for i in test_indices:
    svm_x_train.append(svm_x2_train[i])
    svm_y_train.append(svm_y2_train[i])
svm_y_train

[array([ 0.,  1.,  0.]),
 array([ 0.,  0.,  1.]),
 array([ 0.,  0.,  1.]),
 array([ 0.,  0.,  1.]),
 array([ 0.,  1.,  0.]),
 array([ 0.,  0.,  1.]),
 array([ 0.,  1.,  0.]),
 array([ 0.,  1.,  0.]),
 array([ 0.,  1.,  0.]),
 array([ 0.,  0.,  1.]),
 array([ 0.,  1.,  0.]),
 array([ 0.,  1.,  0.]),
 array([ 0.,  1.,  0.]),
 array([ 0.,  1.,  0.]),
 array([ 0.,  0.,  1.]),
 array([ 1.,  0.,  0.]),
 array([ 0.,  1.,  0.]),
 array([ 0.,  1.,  0.]),
 array([ 0.,  0.,  1.]),
 array([ 0.,  1.,  0.]),
 array([ 1.,  0.,  0.]),
 array([ 0.,  0.,  1.]),
 array([ 0.,  1.,  0.]),
 array([ 0.,  1.,  0.]),
 array([ 0.,  0.,  1.]),
 array([ 0.,  1.,  0.]),
 array([ 0.,  1.,  0.]),
 array([ 0.,  1.,  0.]),
 array([ 1.,  0.,  0.]),
 array([ 0.,  1.,  0.]),
 array([ 0.,  1.,  0.]),
 array([ 1.,  0.,  0.]),
 array([ 0.,  1.,  0.]),
 array([ 1.,  0.,  0.]),
 array([ 0.,  1.,  0.]),
 array([ 0.,  1.,  0.]),
 array([ 0.,  1.,  0.]),
 array([ 1.,  0.,  0.]),
 array([ 1.,  0.,  0.]),
 array([ 0.,  1.,  0.]),


In [16]:
from sklearn import svm

In [32]:
svm_x_test = np.array(svm_x_test)

In [18]:
svm_x_train = np.array(svm_x_train)

In [19]:
clf = svm.SVC(kernel='rbf', class_weight='balanced')

In [20]:
dataset_size = len(svm_x_train)
svm_x_train = np.array(svm_x_train).reshape(dataset_size,-1)

In [21]:
svm_y_train = np.array(svm_y_train)
svm_x_train.shape

(50, 399168)

In [22]:
svm_y_train = [np.where(r==1)[0][0] for r in svm_y_train]
print("test")

test


In [33]:
svm_x_test[0].shape

(399168,)

In [26]:
clf.fit(svm_x_train, svm_y_train)

SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [27]:
svm_y_train

[1,
 2,
 2,
 2,
 1,
 2,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 2,
 0,
 1,
 1,
 2,
 1,
 0,
 2,
 1,
 1,
 2,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 2,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1]

In [28]:
print(x_test[0])
dummy2_y[0]

[[[ 0.          0.          0.        ]
  [ 0.          0.          0.03137255]
  [ 0.          0.          0.        ]
  ..., 
  [ 0.          0.          0.18431373]
  [ 0.          0.          0.09411765]
  [ 0.          0.          0.        ]]

 [[ 0.          0.          0.        ]
  [ 0.          0.          0.03137255]
  [ 0.          0.          0.13333334]
  ..., 
  [ 0.          0.          0.15294118]
  [ 0.          0.          0.09411765]
  [ 0.          0.          0.        ]]

 [[ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.15294118]
  ..., 
  [ 0.          0.          0.05098039]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]]

 ..., 
 [[ 0.73333335  0.          0.36470589]
  [ 0.74509805  0.          0.34901962]
  [ 0.8509804   0.          0.20392157]
  ..., 
  [ 0.96862745  0.16078432  0.        ]
  [ 0.96862745  0.16078432  0.        ]
  [ 0.96862745  0.16078432  0.     

array([ 0.,  0.,  1.])

In [30]:
svm_x_test = []
svm_y_test = []
for i in range(len(x_test)):
    x_1 = np.expand_dims(x_test[i], axis=0)
    #x_1 = preprocess_input(x_1)
    flatten_2_features = model2.predict(x_1)
    svm_x_test.append(flatten_2_features)
    svm_y_test.append(dummy2_y[i])

In [31]:
dataset_size = len(svm_x_test)
svm_x_test = np.array(svm_x_test).reshape(dataset_size,-1)
svm_y_test = [np.where(r==1)[0][0] for r in svm_y_test]
svm_x_test

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.16212584,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ..., 
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]], dtype=float32)

In [45]:
print(svm_y_test[:45])

[2, 1, 1, 1, 1, 1, 2, 1, 1, 2, 2, 0, 2, 1, 1, 2, 1, 2, 0, 2, 0, 2, 2, 1, 1, 1, 2, 1, 1, 1, 0, 1, 1, 1, 0, 0, 2, 2, 1, 1, 0, 1, 2, 1, 1]


In [46]:
svm_x_test.shape

(50, 399168)

In [47]:
print(clf.predict(svm_x_test[:45]))

[2 1 1 1 1 1 2 1 1 2 2 2 2 1 1 2 1 2 0 2 2 2 2 1 1 1 2 1 1 1 2 1 1 1 0 0 2
 2 1 1 2 1 2 1 1]


In [48]:
from sklearn.metrics import accuracy_score
accuracy_score(svm_y_test, clf.predict(svm_x_test))

0.92000000000000004

In [40]:
svm_x_test.shape

(50, 399168)

Right now it is very bad at classifying, I beleive this is because not enough data to make a good feature extracter from the NN. Going to try taking in input from a different layer of the NN to SVM

In [51]:
model2 = Model(inputs=model.input, outputs=model.get_layer('activation_5').output)
svm_x_train = []
svm_y_train = []
for i in range(len(x_new_train)):
    x_1 = np.expand_dims(x_new_train[i], axis=0)
    x_1 = preprocess_input(x_1)
    flatten_2_features = model2.predict(x_1)
    svm_x_train.append(flatten_2_features)
    svm_y_train.append(dummy_y[i])
dataset_size = len(svm_x_train)
svm_x_train = np.array(svm_x_train).reshape(dataset_size,-1)
svm_y_train = [np.where(r==1)[0][0] for r in svm_y_train]
clf.fit(svm_x_train, svm_y_train)

SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [53]:
svm_y_train

[2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]